# Diffe-Hellman

**Problema:** hay poder comunicarnos de forma simetrica para luego comunicarnos de forma asimetrica. Para ello se utiliza un protocolo de intercambio de llaves. 

## Protocolo de intercambio de llaves

Sea $(\mathbf{G}, \dot)$ un grupo cícilico de orden primo $q$ generado por $g \in \mathbf{G}$. El elemento generador es un elemento $g$ de un grupo ciclico $(G, \dot)$ tal que haciendo potencias $g^0, g^1, ...$ genera todos los elementos del grupo.

![diffie-hellman-key-exchange](../img/diffie-hellman-key-exchange.png)

Tanto Alice como Bob tienen $w = g^{\alpha \beta}$ como secreto compartido. 

In [287]:
import math
import random 

from typing import Tuple, List

## Problema del logaritmo discreto

El atacante tiene 
- $g$ que es publico.
- $u = g^\alpha$ y $v = g^\beta$ que estan en el canal.
- El secreto compartido es $w = g^{\alpha \beta}$
- Si hacemos logarimo base $g$, podemos obtener alpha y beta. 
    $$
    log_g(g^\beta) = \beta
    $$

    $$
    log_g(u) = \alpha
    $$

    Sin embargo, esto no es tan fácil, pues estamos trabajando con números discretos en un grupo ciclo.

In [68]:
def gcd(x: int, y: int) -> int: 
    """Algoritmos de eculides""" 

    b, a = (x, y) if x < y else (y, x) 

    while b > 0: 
        r = a % b 
        a, b = b, r

    return a 


def egcd(x: int, y: int) -> Tuple[int, int, int]:
    """Algoritmo de euclides extendido

    Args:
        x (int)
        y (int)

    Returns:
        (r: int, s:int, t:int) tal que s*x + t*y = r = mdc(x, y)
    """
    r2, r1 = (y, x) if x < y else (x, y)

    s1, s2 = 1, 0
    t1, t2 = 0, 1

    while r2 != 0: 
        q = r1 // r2
        r = r1 % r2 

        r1, s1, t1, r2, s2, t2 = r2, s2, t2, r, s1 - s2*q, t1 - t2*q

    return r1, s1, t1


def modinv(a, b):

    # El pequeño teorema de ferman 
    # Si mcd(a, p) = 1
    # a ^ p - 1 = 1 mod p     
    r, s, _ = egcd(a, b)

    if r != 1:
        raise Exception("Inverse doesn't exist")
    
    return (s % b + b) % b

In [37]:
class DiscreteLog:
    
    @staticmethod
    def bruteForce(g: int, y: int, n: int) -> int:
        """Búsqueda Exaustiva.

        Args:
            g (int): generador
            y (int): y = g^x
            n (int): orden del grupo G

        Returns:
            int: log_g(y)
        """
        x = 0
        while pow(g, x, n) != y: 
            x += 1
        return x
    

    @staticmethod
    def babyStep_GigantStep(
            g: int, 
            y: int, 
            n: int, 
            debug_log = False
        ) -> int:
        
        """Búsqueda Exaustiva "más inteligente" introducciendo una tabla de memoria. Se pasa en que dando y = g**x, se puede escribri x = i_x * m + j_x donde 0 <= i_x, j_x. Entonces y = g**{i_x*m}g**{j_x} con m >= raiz(n)

        Args:
            g (int): generador
            y (int): y = g^x
            n (int): orden del grupo G

        Returns:
            int: log_g(y)
        """
        m = math.ceil(math.sqrt(n))
        if debug_log: print("m = ", m)
        
        # lookup es una tabla hash para almacenar tuplas (a, b) \in G X Z
        # a es usada como clave indice
        lookup = {}

        # Computar (g**j, j) para todo 0 <= j <= m
        for j in range(m):
            lookup[pow(g, j, n)] =  j  

        b = pow(g, m * (n - 2), n)
        
        _y = y 
        x = None

        # Computar y * (g**{-m})**i para i = 0, 1, 2, ... hasta entonces i_x con y * (g**{-m})**i_x
        for i in range(m):
            if _y in lookup: 
                x = (i * m + lookup[_y]) % n
                if debug_log: print(f"ix = {i} | jx = {lookup[_y]}")
                return x
            
            _y = (_y * b) % n
        return x 


    @staticmethod
    def pollardsRho(g: int, y: int, n: int, m: int = 3) -> int: 
        pass 

    @staticmethod
    def pohlig_hellman(g: int, y: int, m: int) -> int: 
        pass 

In [4]:
assert(DiscreteLog.bruteForce(3, 57, 113) == 100), "Caso de pre-prueba 1"
assert(DiscreteLog.bruteForce(13, 5732, 38629) == 23718), "Caso de pre-prueba 2"

In [5]:
assert(DiscreteLog.babyStep_GigantStep(3, 57, 113) == 100), "Caso de pre-prueba 1"
assert(DiscreteLog.babyStep_GigantStep(13, 5732, 38629) == 23718), "Caso de pre-prueba 2"

assert(DiscreteLog.babyStep_GigantStep(19, 1249446553, 3129553951) == 972), "Caso de prueba 1"
assert(DiscreteLog.babyStep_GigantStep(19, 1998919620, 3129553951) == 666261645), "Caso de prueba 2"
assert(DiscreteLog.babyStep_GigantStep(19, 1132858435, 3129553951) == 320275002), "Caso de prueba 3"
assert(DiscreteLog.babyStep_GigantStep(19, 2346346458, 3129553951) == 1322464706), "Caso de prueba 4"
assert(DiscreteLog.babyStep_GigantStep(19, 743603545, 3129553951) == 1644500154), "Caso de prueba 5"

## ElGamal

In [443]:
class ElGamal:
    
    def __init__(self, g: int, q: int) -> None:
        self.g = g
        self.q = q


    def G(self) -> Tuple[int, int]:
        a = random.randrange(2, self.q)
        u = pow(self.g, a, self.q)
        return u, a 


    def E(self, pk: int, m: int, b: int = None, debug_log: bool = False):
        b = random.randrange(2, self.q) if b == None else b
        if debug_log: print(f"beta = {b}")

        v = pow(self.g, b, self.q)
        if debug_log: print(f"v = {v}")

        w = pow(pk, b, self.q) 
        c = (m * w) % self.q
        if debug_log: print(f"c = {c}")

        return v, c
    

    def D(self, sk: int, v: int, c: int) -> int:
        w = pow(v, sk, self.q)

        m = c * modinv(w, self.q) % self.q
        return m 

In [407]:
def partition(m: str, chunks_size: int) -> List[int]: 
    return [int.from_bytes(bytes(m[i:i+chunks_size], 'utf-8'), 'big') for i in range(0, len(m), chunks_size)]

def unpartition(c: List[int], chunks_size: int):
    return "".join([ci.to_bytes(chunks_size, 'big').decode("utf-8") for ci in c])

In [412]:
m = '-Life before death.-'
assert(unpartition(partition(m, 4), 4) == m)

In [382]:
def process_encrypt(
        eg: ElGamal,
        pk: int,
        m: int,
        B: List[int] = None
    ) -> None:

    chunks = partition(m, 4)
    B = [None] * len(chunks) if B == None else B 
    C = []

    for i, mi in enumerate(chunks):
        v, c = eg.E(pk, mi, B[i])
        C.append((v, c))
        
    return C 

In [383]:
q = 178286657344291
g = 10
pk = 171079742868778
sk = 104527065844788

EG = ElGamal(g, q)

m = '-Life before death.-'
B=[ 151010711822800,
   27870359061846,
   88930192357307,
   174941820195607,
   42454896251786,
]

process_encrypt(EG, pk, m)
assert(process_encrypt(EG, pk, m, B = B) == [
    (156872974578213,143424279344879),
    (159166700428188,147896452446083),
    (157845678122136,115154487002354),
    (51458276859436,18626216083937),
    (105263726876730,6471583782718)
])


m = 'Strength before weakness'
process_encrypt(EG, pk, m)

B = [10029042278109,68057778179862,138458964750808,79568309796230,32790836980569,126532045980500]

assert(process_encrypt(EG, pk, m, B=B) == [
    (96490178512202, 68603378831044),
    (24077891682173,169748809204785),
    (2085753595936,147200895523904),
    (121494880791917,100454096849250),
    (163816235955194,98752799829277),
    (106521625815058,31822295911609)
])

In [441]:
def process_decrypt(
        eg: ElGamal,
        sk: int,
        C: int,
    ) -> None:

    M = []

    for v, c in C:
        mi = eg.D(sk, v, c)
        M.append(mi)
        
    return unpartition(M, 4)

In [445]:
q = 178286657344291
g = 10
pk = 171079742868778
sk = 104527065844788

C = [
    (141732539323625, 40852371919502),
    (114606755367319, 146664193381344),
    (75283167769359, 112522016284795),
    (165955989847723, 45226837004872),
    (10080996027558, 104523663523835),
    (10165852863804, 114139926880052),
    (55938679345729, 1303889740223)
]

EG = ElGamal(g,q)
assert(process_decrypt(EG, sk, C) == '-Journey before destination-')

True